#### This IPython notebook explores the data for preprocessing. The final data preprocessing will be done in the script file "Data_Preprocess.py".

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r'D:\ML_Projects\AI_Tech_ChatBot\Data\UbuntuDialogueText.csv')

In [5]:
df.head(10)

,folder,dialogueID,date,from,to,text
0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t
4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?
5,3,64545.tsv,2009-08-01T06:22:00.000Z,richardcavell,mechtech,"it's a wikipedia term. There is Prodigy, PRDI..."
6,3,98758.tsv,2009-05-20T07:58:00.000Z,C-00000100,Severity1,what is bartek trying to do?
7,3,98758.tsv,2009-05-20T08:01:00.000Z,C-00000100,Severity1,is he trying to play online video?
8,3,98758.tsv,2009-05-20T08:04:00.000Z,C-00000100,Severity1,"bartek go here, if this doesn't play then it g..."
9,3,86303.tsv,2010-08-24T19:56:00.000Z,pozic,Somelauw,the algorithms from aptitude were ported to ap...


In [5]:
df.shape

(1038324, 6)

In [17]:
df['from'].unique()

array(['bad_image', 'lordleemo', 'mechtech', ..., 'torias', 'alexs',
       'lokpest_'], dtype=object)

In [18]:
df['from'].nunique()

164083

In [6]:
df['to'].unique()

array([nan, 'bad_image', 'mechtech', ..., 'torias', 'alexs', 'lokpest_'],
      dtype=object)

In [16]:
df['to'].nunique()

149488

In [19]:
df['dialogueID'].unique()

array(['126125.tsv', '64545.tsv', '98758.tsv', ..., '19585.tsv',
       '51506.tsv', '99669.tsv'], dtype=object)

In [20]:
df['dialogueID'].nunique()

346108

In [7]:
for i in range(10):
    print(df['text'][i])

Hello folks, please help me a bit with the following sentence: 'Order here your personal photos or videos.' - I think the only allowed version is 'Order your personal videos or photos here.', but I'm not sure, are you?
Did I choose a bad channel? I ask because you seem to be dumb like windows user
the second sentence is better english   and we are not dumb
Sock Puppe?t
WTF?
it's a wikipedia term.  There is Prodigy, PRDIGY and prgidy and they're all the same guy
what is bartek trying to do?
is he trying to play online video?
bartek go here, if this doesn't play then it gives you a link to get it... http://www.milkandcookies.com/link/45466/detail/
the algorithms from aptitude were ported to apt-get, AFAIK.


In [3]:
gpt_chatlogs = pd.read_csv(r'D:\ML_Projects\AI_Tech_ChatBot\Data\ChatGPT_chatlogs\ChatGPT_chatlogs.csv')

In [5]:
gpt_chatlogs.shape

(89311, 3)

In [10]:
gpt_chatlogs.tail(1)

,post_number,system_message,conversation
89310,[22628],[''],"[{'user': 'Anonymous', 'message': 'Can you exp..."


In [16]:
gpt_chatlogs['conversation'].iloc[-1]

'[{\'user\': \'Anonymous\', \'message\': \'Can you explain the SOLID principles in OOP?\'}, {\'user\': \'Chat GPT\', \'message\': \'SOLID is an acronym for five design principles in object-oriented programming (OOP). These principles aim to make software design more understandable, flexible, and maintainable. The SOLID principles are: Single Responsibility Principle (SRP): A class should have only one reason to change, and that reason should be encapsulated within the class. Open-Closed Principle (OCP): A class should be open for extension but closed for modification. Liskov Substitution Principle (LSP): Subtypes should be substitutable for their base types. Interface Segregation Principle (ISP): A class should not be forced to implement interfaces it does not use. Dependency Inversion Principle (DIP): High-level modules should not depend on low-level modules, but both should depend on abstractions. By following these principles, developers can create code that is more robust, easier t